In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os,sys
import errno
import string
from nltk.corpus import reuters 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score,accuracy_score,roc_auc_score,mean_absolute_error
from sklearn.preprocessing import label_binarize
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import NearestCentroid
import csv
from sklearn import svm
import warnings
import random
from statistics import mode
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
random.seed(0)
warnings.filterwarnings('ignore')
def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels
def loadTimeSeriesDatasets(directory, dataset):
    f=directory+"\\"+dataset+"\\"+dataset+'_TEST.arff'
    data = arff.loadarff(f)
    df = pd.DataFrame(data[0])
    test_labels=df['target']
    test_data= np.array(df.loc[:, df.columns != 'class_atr'])
    f=directory+"\\"+dataset+"\\"+dataset+'_TRAIN.arff'
    data = arff.loadarff(f)
    df = pd.DataFrame(data[0])
    train_labels=df['target']
    train_data= np.array(df.loc[:, df.columns != 'class_atr'])
    return train_data, train_labels, test_data, test_labels

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)
def Cosine(a, b):#distance
    return distance.cosine(a,b)
def EISC(doc1,doc2): 
    dot=np.sum(np.sqrt(np.multiply(doc1,doc2)))
    isc=dot /( math.sqrt(np.linalg.norm(doc1, ord=1))*math.sqrt(np.linalg.norm(doc2, ord=1)))
    return 1-isc
def most_common(lst):
    return(mode(lst))


def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=list(range(1,26)),train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    if std==[]:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["k",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=train_time+test_time
    for i in range(0,len(Arr)):
        print(i,len(Arr))
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if std==[]:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if std==[]:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)

def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    averageMeanPrecison=[]
    kList=list(range(1,26))
    func=globals()[metric]
    train_time=[]
    test_time=[]
    maxval=np.max(kList)
    var=np.zeros(train_data.shape[1])
    class_dist=[]
    label_dist=[]
    ##########Main piece of code############
    # split training data according to class distribution
    for k in kList:
        time1=time.time()
        pred=[]
        print("k",k)
        k1=k
        if k1<15:
            k1=k*(len(classes)+1) 
        classifier = KNeighborsClassifier(n_neighbors=k1,metric=func)
        if len(train_labels)<k1:
            k=len(train_labels)
            classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
        classifier.fit(train_data,train_labels)
        distances,neighs=classifier.kneighbors(test_data,return_distance=True)
        distances=distances

        time2=time.time()
        for t in range(0,test_data.shape[0]):
            weights2=[]
            dist=distances[t,:]
            neighs2=neighs[t,:]
            weights2=np.ones((len(dist)))
            if (dist[-1]-dist[0]) !=0:
                weights2 = (dist[-1] - dist)/(dist[-1]-dist[0])
            neigh_data=train_data[neighs[t],:]
            neigh_dist=distances[t,:]
            neigh_labels=[train_labels[n] for n in neighs2]
            weightSum=0
            maxweight=-np.inf
            predLabel=''
            for lbl in set(neigh_labels):
                w3=np.nansum([weights2[j]/(distances[t,j]+0.00001) for j,ii in enumerate(neighs[t]) if train_labels[ii]==lbl])
                weightSum=w3
                if weightSum>=maxweight:
                    maxweight=weightSum
                    predLabel=lbl
            pred.append(predLabel)
        y_pred.append(pred) 
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
          
    class_dict={}
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)

        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)

    #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    
    
import glob
directory = "Time Series Datasets"
datasets = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]
print(datasets)

 
for dataset in datasets:
    print(dataset)
    train_data,train_labels, test_data,test_labels=loadTimeSeriesDatasets("Time Series Datasets\\", dataset)
    labels=list(train_labels).copy()
    labels.extend(test_labels)
    labels=np.array(labels)
    labels=list(labels)
    le = preprocessing.LabelEncoder().fit(labels)
    labels=le.transform(labels)
    train_labels=le.transform(train_labels)
    test_labels=le.transform(test_labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    classes=list(set(labels))
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "EDWKNN\\Time Series Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_EDWKNN.txt'
        tables = open(fname, 'wb')
        tables.write("\n*************".encode())
        tables.write((" Split\t"+str(n_split)).encode())
        tables.write(" ***********\n".encode())
        n_split=n_split+1
        k=1
        print("###############")
        classify(k=k,metric=met)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        tables.close()
        print("###############")





[nltk_data] Downloading package reuters to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Fix-
[nltk_data]     Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['Car', 'Fish', 'GunPoint', 'Ham', 'Haptics', 'Herring', 'InlineSkate', 'Meat', 'Plane', 'Strawberry', 'Trace', 'Yoga']
Car
###############
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
{0: 0, 1: 1, 2: 2, 3: 3}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+----------------------+----------------------+---------------------+
|    k     | Accuracy |      Train Time      |      Test time       |      Total Time     |
+----------+----------+----------------------+----------------------+---------------------+
|    1     |  0.8167  |    0:00:00.065302    |    0:00:00.003990    |    0:00:00.065302   |
|    2     |  0.7667  |    0:00:00.030067    |    0:00:00.005986    |    0:00:00.030067   |
|    3     |  0.7667  |    0:00:00.028923    |    0:00:00.006947    |    0:00:00.0289

k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+----------------------+---------------------+--------------------+
|    k     | Accuracy |      Train Time      |      Test time      |     Total Time     |
+----------+----------+----------------------+---------------------+--------------------+
|    1     |  0.9029  |    0:00:00.250306    |    0:00:00.015990   |   0:00:00.250306   |
|    2     |  0.8686  |    0:00:00.230386    |    0:00:00.021907   |   0:00:00.230386   |
|    3     |  0.8629  |    0:00:00.212431    |    0:00:00.024967   |   0:00:00.212431   |
|    4     |   0.84   |    0:00:00.206415    |    0:00:00.032998   |   0:00:00.206415   |
|    5     |  0.8286  |    0:00:00.204454

###############
GunPoint
###############
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+----------------------+-----------------------+----------------------+
|    k     | Accuracy |      Train Time      |       Test time       |      Total Time      |
+----------+----------+----------------------+-----------------------+----------------------+
|    1     |  0.9333  |    0:00:00.045877    |     0:00:00.007979    |    0:00:00.045877    |
|    2     |  0.9333  |    0:00:00.050865    |     0:00:00.006980    |    0:00:00.050865    |
|    3     |  0.9467  |    0:00:00.044880    |     0:00:00.006981    |    0:00:00.044880    |
|    4     |  0.9133  |    0:00:00.045879    |     0:00:00.008987    |    0:00:00.045879    

k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+---------------------+----------------------+---------------------+
|    k     | Accuracy |      Train Time     |      Test time       |      Total Time     |
+----------+----------+---------------------+----------------------+---------------------+
|    1     |  0.6095  |    0:00:00.087766   |    0:00:00.004987    |    0:00:00.087766   |
|    2     |  0.6095  |    0:00:00.087764   |    0:00:00.006981    |    0:00:00.087764   |
|    3     |  0.6095  |    0:00:00.094747   |    0:00:00.007979    |    0:00:00.094747   |
|    4     |  0.6381  |    0:00:00.084774   |    0:00:00.009973    |    0:00:00.084774   |
|    5     |  0.6571  |    0:00:00.086768   |    0:00:00.009973    |

...

k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
{0: 0, 1: 1, 2: 2}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+--------------------+-----------------------+---------------------+
|    k     | Accuracy |     Train Time     |       Test time       |      Total Time     |
+----------+----------+--------------------+-----------------------+---------------------+
|    1     |   1.0    |   0:00:00.033902   |     0:00:00.002991    |    0:00:00.033902   |
|    2     |   1.0    |   0:00:00.034908   |     0:00:00.002991    |    0:00:00.034908   |
|    3     |   1.0    |   0:00:00.033909   |     0:00:00.004987    |    0:00:00.033909   |
|    4     |   1.0    |   0:00:00.032912   |     0:00:00.004987    |    0:00:00.032912   |
|    5     |   1.0    |   0:00:00.039893   |     0:00:00.006983   

k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+-----------------------+----------------------+---------------------+
|    k     | Accuracy |       Train Time      |      Test time       |      Total Time     |
+----------+----------+-----------------------+----------------------+---------------------+
|    1     |  0.9714  |     0:00:00.087766    |    0:00:00.005985    |    0:00:00.087766   |
|    2     |  0.9619  |     0:00:00.089759    |    0:00:00.008977    |    0:00:00.089759   |
|    3     |  0.9619  |     0:00:00.091754    |    0:00:00.012965    |    0:00:00.091754   |
|    4     |  0.9619  |     0:00:00.089760    |    0:00:00.016955    |    0:00:00.089760   |
|    5     |  0.9619  | 

###############
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+---------------------+----------------------+--------------------+
|    k     | Accuracy |      Train Time     |      Test time       |     Total Time     |
+----------+----------+---------------------+----------------------+--------------------+
|    1     |  0.9946  |    0:00:01.933830   |    0:00:00.018950    |   0:00:01.933830   |
|    2     |  0.9973  |    0:00:01.982700   |    0:00:00.024932    |   0:00:01.982700   |
|    3     |  0.9973  |    0:00:01.974722   |    0:00:00.029922    |   0:00:01.974722   |
|    4     |  0.9973  |    0:00:01.989680   |    0:00:00.032912    |   0:00:01.989680   |
|    5     |  0.9973  |    0:00:02.045533   |    0:

k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
{0: 0, 1: 1, 2: 2, 3: 3}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+----------------------+----------------------+----------------------+
|    k     | Accuracy |      Train Time      |      Test time       |      Total Time      |
+----------+----------+----------------------+----------------------+----------------------+
|    1     |   0.79   |    0:00:00.092751    |    0:00:00.007979    |    0:00:00.092751    |
|    2     |   0.71   |    0:00:00.093749    |    0:00:00.008977    |    0:00:00.093749    |
|    3     |   0.68   |    0:00:00.095744    |    0:00:00.011968    |    0:00:00.095744    |
|    4     |   0.66   |    0:00:00.104720    |    0:00:00.012965    |    0:00:00.104720    |
|    5     |   0.63   |    0:00:00.093

###############
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 25
16 25
17 25
18 25
19 25
20 25
21 25
22 25
23 25
24 25
+----------+----------+--------------------+--------------------+-------------------+
|    k     | Accuracy |     Train Time     |     Test time      |     Total Time    |
+----------+----------+--------------------+--------------------+-------------------+
|    1     |  0.8467  |   0:00:08.904200   |   0:00:00.264295   |   0:00:08.904200  |
|    2     |  0.8457  |   0:00:08.604726   |   0:00:00.300198   |   0:00:08.604726  |
|    3     |  0.8357  |   0:00:08.603714   |   0:00:00.378987   |   0:00:08.603714  |
|    4     |  0.8233  |   0:00:08.657859   |   0:00:00.556512   |   0:00:08.657859  |
|    5     |   0.8    |   0:00:09.194934   |   0:00:00.517616   |   0:00:09.194

In [2]:
test=[1,2,3,4]
print(test[0:-1])

[1, 2, 3]


In [3]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
y_scores = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
roc_auc_score(y_true, y_scores)

0.5